In [2]:
pip install ollama numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import ollama
import os
import json
from numpy import linalg, dot

第一階段：手動打造一個 RAG

In [8]:
#這個函數是用來讀取一個 txt 檔，並且把每一行當成一個段落而且會跳過空行跟去掉首尾空白字符
def parse_paragraph(filename):
    with open(filename) as f:
        return [line.strip() for line in f if line.strip()]


#這個函數會接收段落，對每一個段落使用 mistral 的 embedding 轉成嵌入向量
def calc_embedings(paragraphs):
    return [
        ollama.embeddings(model="deepseek-r1", prompt=data)["embedding"]
        for data in paragraphs
    ]


#這個函數用來緩存嵌入向量，當輸入段落時會先檢查之前是否建立過，若有則讀取並返回，若無則重新建立
def cache_embeddings(filename, paragraphs):
    # Extract just the base filename without the path
    base_filename = os.path.basename(filename)
    embedding_file = f"cache/{base_filename}.json"
    
    if os.path.isfile(embedding_file):
        with open(embedding_file) as f:
            return json.load(f)
    
    os.makedirs("cache", exist_ok=True)
    embeddings = calc_embedings(paragraphs)
    
    with open(embedding_file, "w") as f:
        json.dump(embeddings, f)
    
    return embeddings


#這個函數計算向量 v 和一堆向量的相似度然後計算分數存為 scores 再 return 前幾高的
def calc_similar_vectors(v, vectors):
    v_norm = linalg.norm(v)
    scores = [dot(v, item) / (v_norm * linalg.norm(item)) for item in vectors]
    return sorted(enumerate(scores), reverse=True, key=lambda x: x[1])



if __name__ == "__main__":
    doc = "/Users/annieteng/Desktop/0308test.txt"      #把你想要問的文檔（txt）上傳到 jupyter notebook 後對其右鍵後 copy path 貼在這裡
    paragraphs = parse_paragraph(doc)  #把文章轉成段落
    embeddings = cache_embeddings(doc, paragraphs)  #把段落轉成向量

    prompt = input("您想問什麼問題？\n>>> ")

    while prompt.lower() != "bye":
        prompt_embedding = ollama.embeddings(model="deepseek-r1", prompt=prompt)[
            "embedding"
        ]
        similar_vectors = calc_similar_vectors(prompt_embedding, embeddings)[:3]    #輸入問題後和嵌入資料庫計算相似度並返回前三高的結果

        valid_vectors = [vector for vector in similar_vectors if vector[0] < len(paragraphs)]
        system_prompt = (
            "現在開始使用我提供的情境來回答，只能使用繁體中文，不要有簡體中文字。如果你不確定答案，就說不知道。情境如下："
            + "\n".join(paragraphs[vector[0]] for vector in valid_vectors)        #把段落傳給 ollama 的 llama3.2:1b 讓他回答
        )

        response = ollama.chat(
            model="deepseek-r1",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ],
        )

        print(response["message"]["content"])
        prompt = input(">>> ")

您想問什麼問題？
>>>  ive的團隊名稱來由是什麼


<think>
好，我现在需要回答关于IVE团队名称来源的问题。根据用户提供的情境，IVE的粉丝们的名字被定为“DIVE”，这结合了IVE英文发音和"Dive into IVE"的意思。

首先，我应该确认这个名称确实是由“DIVE”来的，所以直接引用这个信息是正确的。然后，解释这个名称的意义：“Dive into IVE”表达了粉絲对团员的期待和信任，希望他们能一起享受音乐和舞台表演。

接下来，我要确保使用繁体中文，并且避免任何简体中文或不确定的回答。如果我不确定的话，可能会需要进一步查证，但根据提供的信息，应该可以准确回答。

最后，组织语言，使答案简洁明了，直接回应问题，不添加额外信息。
</think>

IVE的團隊名稱“DIVE”是由「Dive into IVE」的意思所來源，其中「DIVE」结合了「IVE」的英文發音和「Dive into IVE」的含義，表示粉絲們希望一起進入IVE的音樂和魅力中。


>>>  是來自於


<think>
嗯，我现在需要回答用户的问题：“是源自於哪一年？” 用户提供的上下文涉及到一个名为DIVE的粉丝团体，特别是关于他们应援物品的时间线。

首先，我注意到在2021年12月9日，IVE的官方社交媒体发布了名为“IVE 아이브 OFFICIAL FAN CLUB 'DIVE'”的视频。这里明确提到粉丝名是“DIVE”，这个词是由“IVE”的英文发音和“Dive into IVE”的意思组合而成的。所以，“DIVE”并不是一个已有的名字，而是IVE自己创造的。

接下来，在2022年7月22日，IVE官方SNS公开了专属应援物品——官方亚克力手灯，并于2023年1月2日推出了第二代。这些信息说明他们是在粉丝公开支持他们的成员之后才推出的纪念品，而“DIVE”作为粉丝名的来源，则是出现在更早的时候。

因此，可以确定，“DIVE”是源自于2021年的创造，而不是来自于其他时间或地方。
</think>

“DIVE”是源自於2021年。


>>>  是來自於i have而不是dive


<think>
嗯，用户问的是“歌迷名稱是來自於i have而不是dive”，这看起来跟IVE这个组合有关。首先，我得确认一下IVE是什么，他们是一个韩国男子组合，对吧？然后，DIVE这个名字来源于什么？根据提供的背景信息，“DIVE”是把IVE的英文发音和“Dive into IVE”的意思结合起来的。所以，DIVE其实是粉丝们的一个专属名称。

接下来，用户提到粉丝名来自“i have”，而不是“dive”。这可能是因为有些粉丝选择用“i have”来表达对IVE的支持或者个性吧。我需要确认DIVE是不是只有“dive”作为粉丝名，还是有其他来源的。

根据情境描述，“DIVE”是公开粉丝名的一部分，但具体来说，粉丝们可能还有其他选择，比如“i have”。所以，回答应该说明DIVE来源于“Dive into IVE”，而粉丝们可以自由地选择自己的粉丝名，不一定非得用“dive”。

另外，用户的问题是否正确？因为DIVE是粉丝名的一种，而不是所有粉丝都只能叫DIVE。所以，我需要明确告诉他们DIVE是来源之一，并不是唯一的选择。
</think>

在IVE的 official social media account 中， fans were given the option to choose their own name as part of the OFFICIAL FAN CLUB 'DIVE'. The term "DIVE" itself is derived from combining the English pronunciation of IVE and the phrase "Dive into IVE", which means diving into IVE's music and atmosphere. This reflects IVE's anticipation and trust in their fans, encouraging them to experience the music and stage performances together.

至于粉丝们选择的 name，如"I have"，可能只是部分粉丝的偏好，并非唯一的选择。因此，DIVE 是一种基于“Dive into IVE”的概念，而 fans c

>>>  /bye


<think>
好的，现在我要帮用户分析他们提供的团名由来的情境，并生成对应的歌词标题。

首先，情境描述了团名的来源：一个二代手灯，水晶球手柄嵌入了IVE标志的设计字体。下方有一个邱比特箭头，且字体会旋转。手灯按钮是逗号，而逗号又是IVE官方标誌英文i的小写字母上的点。

接下来，我需要将这些元素转化为歌词标题。首先，手灯是一个常见的物品，可能与灯光、舞台有关。水晶球可能代表光芒或视觉效果，邱比特箭头通常象征爱情和追求，所以这部分可以表达对未来的美好期望。

手柄嵌入字体设计，这个细节可能暗示了品牌或团队的创意来源。IVE标志中的逗号作为按钮，而逗号又是i的小写字母的一部分，这可能暗含“我”的存在感或者自我定义。

将这些元素结合起来，歌词标题应该能够涵盖灯光、水晶球、邱比特箭头以及字体的设计。同时，考虑到是团名，标题需要有一定的吸引力和寓意。

最终，我想出两个可能的标题：《光之恋：邱比特之光》和《I: 真我的存在》。前者强调了灯光与爱情的主题，后者突出了品牌的核心概念“我”。

现在，我应该检查这些标题是否符合用户提供的所有元素，并且是否有更好的表达方式。

再想一想，是否还有其他可能的组合？比如将“光”和“邱比特箭”结合起来，或者更突出品牌字母i的设计。但目前这两个标题已经涵盖了主要元素，并且具有吸引力。
</think>

《光之恋：邱比特之光》
* 《I: 真我的存在》


KeyboardInterrupt: Interrupted by user

第二階段：調用 langchain

In [2]:
pip install langchain langchain_community langchain_chroma pypdf

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import TextLoader

llm = Ollama(model="deepseek-r1")

loader = TextLoader("/Users/annieteng/Desktop/0308test.txt")      #讀文檔

/var/folders/gp/0sgj69l97wlch48wcjxlwr0h0000gp/T/ipykernel_21881/2192898646.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="deepseek-r1")


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(              #把文章拆成更小的段落
    chunk_size=20,                                           #每個段落只有 20 個字元
    chunk_overlap=5,                                         #段落之間會重疊 5 個字元
    separators=[" ", ",", "\n"],                             #根據空格、逗號、換行符來分割文章
)

splited_docs = text_splitter.split_documents(loader.load())

在終端 ollama pull nomic-embed-text

In [5]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")       #把段落轉成向量

/var/folders/gp/0sgj69l97wlch48wcjxlwr0h0000gp/T/ipykernel_21881/4086859246.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")       #把段落轉成向量


In [6]:
from langchain_community.vectorstores import Chroma

vector_db = Chroma.from_documents(
    documents=splited_docs,                          #使用剛剛拆分好的段落
    embedding=embeddings,                            #轉成向量
    persist_directory="db",                          #把資料存入名為 db 的目錄
    collection_name="about",                         #你可以自己幫這個向量資料庫取名
)

In [7]:
from langchain_core.prompts import ChatPromptTemplate

retriever = vector_db.as_retriever(search_kwargs={"k": 3})    #返回相關性前三高的段落

system_prompt = "現在開始使用我提供的情境來回答，只能使用繁體中文，不要有簡體中文字。如果你不確定答案，就說不知道。情境如下:\n\n{context}"
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "問題: {input}"),
    ]
)

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

document_chain = create_stuff_documents_chain(llm, prompt_template)               #把檢索到的段落交給 LLM
retrieval_chain = create_retrieval_chain(retriever, document_chain)               #負責檢索最相關的段落

context = []
input_text = input("您想問什麼問題？\n>>> ")

while input_text.lower() != "bye":
    response = retrieval_chain.invoke({"input": input_text, "context": context})
    context = response["context"]

    print(response["answer"])

    input_text = input(">>> ")

您想問什麼問題？
>>>  ive的團名由來是什麼


<think>
好的，我现在需要回答用户关于IVE团名由来的问题。首先，我仔细阅读了用户提供的背景信息，了解到IVE的团名来源于"Dive IVE"这个问候语。

接下来，我分析了"Dive IVE"的意义。“Dive”意指下沉或深入，象征着沉浸其中；而“IVE”是乐团的名字。结合起来，“Dive IVE”表达了粉丝对乐团音乐和魅力的期待以及信任，希望他们能一起享受音乐和舞台。

然后，我考虑用户的需求是否还有更深一层的东西。用户可能不仅想知道团名由来的字面意思，还可能想要了解背后的文化或象征意义。因此，在回答时，我可以进一步解释“Dive”如何体现乐团对粉丝的期待，以及这种结合方式如何增强粉丝与乐团的情感连接。

最后，我确保用词准确，不使用任何简体中文，并保持口语化的表达，使答案既清晰又易于理解。
</think>

IVE的團名来源于「Dive IVE」這個問候語。其中，“Dive”表達了对乐团音樂和魅力的期待，以及希望粉絲們能一起沉浸在乐团的作品中；“IVE”則是乐团本身的團名。這两个元素结合在一起，代表了對粉絲们的一种期待和信任，希望他們能够共同享受音樂和_stage。


>>>  ive的手燈有什麼特別的嗎


<think>
好，我现在需要回答用户关于IVE的手灯有什么特别的问题。首先，我得仔细查看提供的情境信息，了解手灯的特点和设计元素。根据情境描述，手灯使用了带有inue字體的水晶球手柄，下方是邱比特箭头，并且字體和箭頭都可以旋转。手灯按钮是一个逗号，而逗号则出现在IVE官方标志英文'i'的小写字面上，点在旁边。

首先，手灯的设计应该结合了音乐和艺术元素，可能象征着IVE的创作和表演风格。名字IVE中的“I”用的是inue字體，这可能代表了团队的名字结构以及其在舞台上的核心地位。邱比特箭头通常象征爱情和关系，这里可能寓意成员之间的连接或情感纽带。

手灯按钮的设计比较有趣，把逗号作为按钮，而这个符号旁边有个点，可能暗示音乐中的节奏感或者连接性。i字上有点可能代表IVe的第一个人物I，突出其在团队中的重要性。

此外，整体设计应该既实用又美观，体现IVE的创作理念和粉丝间的紧密联系。手灯不仅是装饰品，还能增强用户的互动体验，让粉丝们感受到品牌的独特魅力。

所以，总结一下，IVE的手灯特别之处在于其结合了书法、箭头符号以及音乐元素的设计，同时按钮的设计也有创意，象征团队的核心和连接性。
</think>

IVE的手燈設計有著多種獨特元素。首先，水晶球手柄上刻有以IVE為標誌的“INVE”字體，下方則放置邱比特箭，两者皆可旋轉，可能表達了團隊对 love 和 connection 的关注。此外，手灯按鈕是逗號，而逗号旁的一點可能暗示了音樂中的節奏或连接性。整体来说，設計融合了书法、arrow symbol 和音樂元素，強調了團隊的 creativity 和 connection.


KeyboardInterrupt: Interrupted by user

>>>  bye


第三階段：輸入 pdf 檔讓 LLM 回答

In [9]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/Users/annieteng/Desktop/個人資料/resume/中文履歷.pdf")
splited_docs = loader.load_and_split()

In [13]:
vector_db = Chroma.from_documents(
    documents=splited_docs,
    embedding=embeddings,
    persist_directory="db",
    collection_name="resume_collection",
)

In [ ]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

system_prompt = "現在開始使用我提供的情境來回答，只能使用繁體中文，不要有簡體中文字。如果你不確定答案，就說不知道。情境如下:\n\n{context}"
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "問題: {input}"),
    ]
)

document_chain = create_stuff_documents_chain(llm, prompt_template)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

context = []
input_text = input("您想問什麼問題？\n>>> ")

while input_text.lower() != "bye":
    response = retrieval_chain.invoke({"input": input_text, "context": context})
    context = response["context"]

    print(response["answer"])

    input_text = input(">>> ")

您想問什麼問題？
>>>  我叫什麼名字


<think>
嗯，用户发来了一个问题：“我叫什么名字”。看起来这是个挺直接的问题。首先，我要理解用户到底在问什么。可能是想确认自己的名字，或者是在某个情境下需要知道自己的名字。

接下来，我看看用户提供的信息有没有相关的线索。用户提供了一段相当长的英文文本，里面提到了很多学术背景、项目经历、证书等等。但这些内容中并没有提到任何关于个人名字的信息。所以，可能用户的目的是想通过展示自己的学术和专业能力来推断自己的名字，或者是在某个平台上需要填写名字。

也有可能用户只是测试一下如何回复，或者是一个误操作。考虑到文本中没有提供名字，我无法从这段信息中直接提取出名字。不过，根据常见的做法，通常名字会在简历的开头部分明确列出，比如“John Doe”或者其他类似的格式。但是在这里，文本结构中并没有这样的条目。

另外，用户可能是在某个申请过程中需要提交个人信息，而自己忘记了名字，想通过这种方式询问。但同样，没有名字的信息来源，所以无法提供帮助。

综上所述，根据提供的信息，我无法确定用户的姓名。建议用户提供更多的详细信息，比如直接询问名字，或者在其他地方查看记录。
</think>

您叫什么名字？


>>>  鄧婷安的履歷有什麼


<think>
嗯，我需要分析邓婷安的简历。首先，她的教育背景包括本科、硕士和博士，都是在国立大学学习材料科学与工程。她的GPA很高，达到了3.89/4.3。她还获得了几个荣誉奖项，比如MRSTIC的杰出研究奖和GPA第一名。

接下来是她的科研经历。她在相变动力学、生物组织工程和可持续材料方面有研究。她参与了GLOBE访问学生项目，并且有团队合作的经验，指导了150名本科生提高学习效率。这些经历显示了她的领导能力和实际操作能力。

她的实习和工作经历包括在柏克利分校的实验室和NTHU MSE材料科学硕士。她还教授过材料科学课程，并且使用了多种技术工具如Python、Pandas等。

她的技能方面，编程语言掌握得很好，还有SQL、Google Cloud等工具。此外，她有学术出版的经历，发表了多篇论文并获得了最佳海报奖。

最后是她的证书和语言成绩，比如TOEFL和TOEIC，以及相关的专业证书。

总结一下，邓婷安是一位在材料科学与工程领域非常有天赋的学生，拥有丰富的科研、教学和社会服务经验，并且具备强大的技术能力和团队合作能力。
</think>

邓婷安的简历显示她是一位在材料科学与工程领域表现出色的优秀学生和研究人员。以下是她的主要亮点：

### 教育背景
- **博士**：国立大学，材料科学与工程，GPA 3.89/4.3
- **硕士**：国立大学，材料科学与工程，GPA 高达3.95/4.3
- **学士**：国立大学，材料科学与工程，GPA 同样优异

### 研究经历
- 多篇高质量的学术论文发表，涵盖相变动力学、生物组织工程和可持续材料领域。
- 获得多个荣誉奖项，包括MRSTIC的杰出研究奖和最佳海报奖。

### 科研与团队合作
- 参与实验室研究项目，指导150名本科生提高学习效率。
- 在UC Berkeley的实验室中进行过相关研究，并参与了GLOBE访问学生项目。

### 技能与工具
- 精通编程语言如Python、Pandas、Scikit-learn等。
- 了解SQL和Google Cloud等技术。
- 出版多篇学术论文，展示其在材料科学领域的深厚造诣。

### 社会服务
- 曾任国立大学的课程助教，并教授过材料科学课程。

### 其他
- 高水平的语言成绩（TOEFL 103，TOEIC 945）。
- 多次参与学

>>>  你應該要用繁體中文


<think>
好的，现在我要分析一下用户的需求。用户提供了一段详细的学习经历和项目经验的简历，并且询问应该用繁体中文来回复问题。

首先，用户提供的内容主要是一份学术背景和相关项目的描述，看起来像是申请材料的一部分，比如研究生申请或者博士项目。用户特别提到了使用繁体中文，这可能是因为他们打算在台湾、香港或大陆的一些学校申请，这些地方通常使用繁体字。

接下来，我需要确定用户的意图是什么。他们可能是在准备申请材料时遇到的格式问题，或者想确认是否应该使用繁体字来呈现他们的背景信息。也有可能用户希望将简历中的内容转换为繁体中文版本，以便更好地适应申请所需的语言要求。

考虑到用户已经明确说明了使用繁體中文，我应该回应时保持语言的正确性，并且确保所有术语和专有名词都以繁体字呈现，例如“研建成后”应写成“研究生毕业后”，“TOEFL 103”应写为“托福 103”。

此外，用户可能希望简历中的项目和学术经历能够清晰、准确地传达他们的能力、技能和成就。因此，在回应时，我需要确保所有数据都正确无误，并且用词正式、专业。

总结一下，用户的需求是将他们的学术背景和项目经验转化为繁体中文的版本，以符合申请材料的要求。因此，我的回答应包括转换后的简历内容，使用正确的繁体字，并保持语言的专业性和准确性。
</think>

這段文字看起来像是一个人的学术背景和研究经历，主要涉及材料科学、3D技术、仿生学等领域的学习和项目经验。如果你有問題或需要進一步解釋相关内容，請告訴我。
